import libraries(I provide all libs that I need when make this tasks, if you need some external import them here)

In [2]:
import findspark
findspark.init('C:\Program Files\spark-3.5.0-bin-hadoop3')

In [3]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col
from pyspark.sql.functions import max, avg, min, round
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, desc
from pyspark.sql.functions import when

create local SparkSession

In [4]:
spark = SparkSession.builder.getOrCreate()

read csv with inferschema

In [4]:
data = spark.read.csv('ds_salaries.csv', inferSchema=True, header=True)

read csv one more time with the same code and you will see that it almostly don't take time, because info already in SparkSession and it will not read nothing
from this file

In [5]:
df = spark.read.csv('ds_salaries.csv', inferSchema=True, header=True)

write schema of scv on screen

In [6]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



In [7]:
df = df.withColumnRenamed('_c0', 'id').show()

+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
| id|work_year|experience_level|employment_type|           job_title|  salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|  0|     2020|              MI|             FT|      Data Scientist|   70000|            EUR|        79833|                DE|           0|              DE|           L|
|  1|     2020|              SE|             FT|Machine Learning ...|  260000|            USD|       260000|                JP|           0|              JP|           S|
|  2|     2020|              SE|             FT|   Big Data Engineer|   85000|            GBP|       109024|                GB|          50|     

create schema of this scv

In [5]:
schema = StructType([
    StructField('id', IntegerType(), True),
    StructField('work_year', IntegerType(), True),
    StructField('experience_level', StringType(), True),
    StructField('employment_type', StringType(), True),
    StructField('job_title', StringType(), True),
    StructField('salary', IntegerType(), True),
    StructField('salary_currency', StringType(), True),
    StructField('salary_in_usd', IntegerType(), True),
    StructField('employee_residence', StringType(), True),
    StructField('remote_ratio', IntegerType(), True),
    StructField('company_location', StringType(), True),
    StructField('company_size', StringType(), True)
             ])

restart kernel without cleaning output and after restarting you need to initialize SparkSession, after initialize start execute only cells from cell with schema=
=StructType.... 
To restart kernel click Kernel, Restart.

read ds_salaries with predefined schema and compare results from this cell and cell with inferSchema

In [7]:
df2 = spark.read.option("header", "true").schema(schema).csv('ds_salaries.csv')

In [8]:
df2.show()

+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
| id|work_year|experience_level|employment_type|           job_title|  salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|  0|     2020|              MI|             FT|      Data Scientist|   70000|            EUR|        79833|                DE|           0|              DE|           L|
|  1|     2020|              SE|             FT|Machine Learning ...|  260000|            USD|       260000|                JP|           0|              JP|           S|
|  2|     2020|              SE|             FT|   Big Data Engineer|   85000|            GBP|       109024|                GB|          50|     

this happens because read operation is lazy(transformation), but if you use inferschema it start to be action that will create Spark Job, because Spark need to loop throw all file to check datatypes for all columns and this can harm to your code(if we compare to parquet, it will also go to check data types, but parquet provide meta information, so Spark will not go throw all file, he will just read meta information, but csv don't provide such meta information). Also header make Spark to create one more Spark Job to check first line
to define name of columns and remember to skeep it when reading. Actual reading start when you will use first action. More about Spark Jobs you will see in next topic

write schema of scv on screen one more time and compare with previous

In [9]:
df2.printSchema()

root
 |-- id: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



now continue to work with one of the dataframes that you create

print data in dataframe using df.show

In [10]:
df2.show()

+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
| id|work_year|experience_level|employment_type|           job_title|  salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|  0|     2020|              MI|             FT|      Data Scientist|   70000|            EUR|        79833|                DE|           0|              DE|           L|
|  1|     2020|              SE|             FT|Machine Learning ...|  260000|            USD|       260000|                JP|           0|              JP|           S|
|  2|     2020|              SE|             FT|   Big Data Engineer|   85000|            GBP|       109024|                GB|          50|     

print data in dataframe using display(df.toPandas())

In [11]:
df2.toPandas()

,id,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L
...,...,...,...,...,...,...,...,...,...,...,...,...
602,602,2022,SE,FT,Data Engineer,154000,USD,154000,US,100,US,M
603,603,2022,SE,FT,Data Engineer,126000,USD,126000,US,100,US,M
604,604,2022,SE,FT,Data Analyst,129000,USD,129000,US,0,US,M
605,605,2022,SE,FT,Data Analyst,150000,USD,150000,US,100,US,M


create df_job_title that consists from all job_titles without duplicates

In [12]:
df_job_title = df2.select('job_title').distinct()

print all rows from df_job_titles without truncating jobs

In [26]:
df_job_title.show(truncate=False)

+----------------------------------------+
|job_title                               |
+----------------------------------------+
|3D Computer Vision Researcher           |
|Lead Data Engineer                      |
|Head of Machine Learning                |
|Data Specialist                         |
|Data Analytics Lead                     |
|Machine Learning Scientist              |
|Lead Data Analyst                       |
|Data Engineering Manager                |
|Staff Data Scientist                    |
|ETL Developer                           |
|Director of Data Engineering            |
|Product Data Analyst                    |
|Principal Data Scientist                |
|AI Scientist                            |
|Director of Data Science                |
|Machine Learning Engineer               |
|Lead Data Scientist                     |
|Machine Learning Infrastructure Engineer|
|Data Science Engineer                   |
|Machine Learning Manager                |
+----------

create  df_analytic that will consists from max, avg, min USD salaries for all job_titles using groupBy. name of fields is avg_salary, min_salary, max_salary

In [15]:
schema = StructType([
    StructField('job_title', IntegerType(), True),
    StructField('avg_salary', IntegerType(), True), 
    StructField('min_salary', IntegerType(), True),
    StructField('max_salary', IntegerType(), True)
  ])

In [44]:
max_sal = df2.groupBy('job_title').max('salary_in_usd').withColumnRenamed('max(salary_in_usd)', 'max_salary_in_usd')
avg_sal = df2.groupBy('job_title').avg('salary_in_usd').withColumnRenamed('avg(salary_in_usd)', 'avg_salary_in_usd')
min_sal = df2.groupBy('job_title').min('salary_in_usd').withColumnRenamed('min(salary_in_usd)', 'min_salary_in_usd')

In [45]:
avg_sal.show(truncate=False)

+----------------------------------------+------------------+
|job_title                               |avg_salary_in_usd |
+----------------------------------------+------------------+
|3D Computer Vision Researcher           |5409.0            |
|Lead Data Engineer                      |139724.5          |
|Head of Machine Learning                |79039.0           |
|Data Specialist                         |165000.0          |
|Data Analytics Lead                     |405000.0          |
|Machine Learning Scientist              |158412.5          |
|Lead Data Analyst                       |92203.0           |
|Data Engineering Manager                |123227.2          |
|Staff Data Scientist                    |105000.0          |
|ETL Developer                           |54957.0           |
|Director of Data Engineering            |156738.0          |
|Product Data Analyst                    |13036.0           |
|Principal Data Scientist                |215242.42857142858|
|AI Scie

In [46]:
df_analytic = max_sal.join(avg_sal, on='job_title', how='inner').join(min_sal, on='job_title', how='inner')

In [48]:
df_analytic = df_analytic.withColumn('avg_salary_in_usd', round('avg_salary_in_usd', 2))

print all rows from df_analytic without trancating jobs

In [49]:
df_analytic.show(truncate=False)

+----------------------------------------+-----------------+-----------------+-----------------+
|job_title                               |max_salary_in_usd|avg_salary_in_usd|min_salary_in_usd|
+----------------------------------------+-----------------+-----------------+-----------------+
|3D Computer Vision Researcher           |5409             |5409.0           |5409             |
|Lead Data Engineer                      |276000           |139724.5         |56000            |
|Head of Machine Learning                |79039            |79039.0          |79039            |
|Data Specialist                         |165000           |165000.0         |165000           |
|Data Analytics Lead                     |405000           |405000.0         |405000           |
|Machine Learning Scientist              |260000           |158412.5         |12000            |
|Lead Data Analyst                       |170000           |92203.0          |19609            |
|Data Engineering Manager     

now you need to add in df_analytic column row_id, that will show order of all job_titles depending on avg salary. they should be descending

In [50]:
window_spec = Window.orderBy(desc('avg_salary_in_usd'))
df_analytic = df_analytic.withColumn('row_id', row_number().over(window_spec))

print all data from df_analytic

In [51]:
df_analytic.show(truncate=False)

+----------------------------------+-----------------+-----------------+-----------------+------+
|job_title                         |max_salary_in_usd|avg_salary_in_usd|min_salary_in_usd|row_id|
+----------------------------------+-----------------+-----------------+-----------------+------+
|Data Analytics Lead               |405000           |405000.0         |405000           |1     |
|Principal Data Engineer           |600000           |328333.33        |185000           |2     |
|Financial Data Analyst            |450000           |275000.0         |100000           |3     |
|Principal Data Scientist          |416000           |215242.43        |148261           |4     |
|Director of Data Science          |325000           |195074.0         |130026           |5     |
|Data Architect                    |266400           |177873.91        |90700            |6     |
|Applied Data Scientist            |380000           |175655.0         |54238            |7     |
|Analytics Engineer 

it isn't beautifull, so we need to put now row_id on first place in df_analytic

In [53]:
df_analytic =  df_analytic.selectExpr('row_id', 'job_title',
                                     'max_salary_in_usd', 'avg_salary_in_usd',
                                     'min_salary_in_usd')

print df_analytic now

In [54]:
df_analytic.show(truncate=False)

+------+----------------------------------+-----------------+-----------------+-----------------+
|row_id|job_title                         |max_salary_in_usd|avg_salary_in_usd|min_salary_in_usd|
+------+----------------------------------+-----------------+-----------------+-----------------+
|1     |Data Analytics Lead               |405000           |405000.0         |405000           |
|2     |Principal Data Engineer           |600000           |328333.33        |185000           |
|3     |Financial Data Analyst            |450000           |275000.0         |100000           |
|4     |Principal Data Scientist          |416000           |215242.43        |148261           |
|5     |Director of Data Science          |325000           |195074.0         |130026           |
|6     |Data Architect                    |266400           |177873.91        |90700            |
|7     |Applied Data Scientist            |380000           |175655.0         |54238            |
|8     |Analytics En

here you need to create df_exp_lvl with the biggest usd_salary(biggest_salary) for each experience_level(you need to save all fields like in entire dataframe)

In [55]:
windowSpecAgg  = Window.partitionBy('experience_level')
df_exp_lvl = df2.withColumn('biggest_salary', max(col('salary_in_usd')).over(windowSpecAgg))

print here df_exp_lvl

In [59]:
df_exp_lvl.select('experience_level', 'biggest_salary').groupBy('experience_level').max().show()

+----------------+-------------------+
|experience_level|max(biggest_salary)|
+----------------+-------------------+
|              EN|             250000|
|              EX|             600000|
|              MI|             450000|
|              SE|             412000|
+----------------+-------------------+



In [62]:
df_exp_lvl.show()

+---+---------+----------------+---------------+--------------------+-------+---------------+-------------+------------------+------------+----------------+------------+--------------+
| id|work_year|experience_level|employment_type|           job_title| salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|biggest_salary|
+---+---------+----------------+---------------+--------------------+-------+---------------+-------------+------------------+------------+----------------+------------+--------------+
|  5|     2020|              EN|             FT|        Data Analyst|  72000|            USD|        72000|                US|         100|              US|           L|        250000|
| 10|     2020|              EN|             FT|      Data Scientist|  45000|            EUR|        51321|                FR|           0|              FR|           S|        250000|
| 12|     2020|              EN|             FT|      Data Scientist|  3500

create df_best that consists from rows where salary of guy same as biggest salary for other people in his exp_lvl and choose only columns: id, experience_level, biggest_salary, employee_residence

In [64]:
df_best = df_exp_lvl.filter(df_exp_lvl.salary_in_usd == df_exp_lvl.biggest_salary).select('id',
                                                                                                'experience_level', 
                                                                                                'biggest_salary',
                                                                                                'employee_residence')

print df_best

In [65]:
df_best.show()

+---+----------------+--------------+------------------+
| id|experience_level|biggest_salary|employee_residence|
+---+----------------+--------------+------------------+
| 37|              EN|        250000|                US|
|252|              EX|        600000|                US|
| 33|              MI|        450000|                US|
| 97|              MI|        450000|                US|
| 63|              SE|        412000|                US|
+---+----------------+--------------+------------------+



drop duplicates if exist by experience_level

In [71]:
df_best = df_best.drop_duplicates(['experience_level'])

print df_best

In [72]:
df_best.show()

+---+----------------+--------------+------------------+
| id|experience_level|biggest_salary|employee_residence|
+---+----------------+--------------+------------------+
| 37|              EN|        250000|                US|
|252|              EX|        600000|                US|
| 33|              MI|        450000|                US|
| 63|              SE|        412000|                US|
+---+----------------+--------------+------------------+



create df_new_best from df_best without id, and make the next: when exp_level = MI we want middle, when SE we want senior, else Null

In [78]:
levels = {'MI': 'middle', 'SE': 'senior', 'EN': None, 'EX': None}
df_new_best = df_best.drop('id').replace(levels)

print df_new_best

In [79]:
df_new_best.show()

+----------------+--------------+------------------+
|experience_level|biggest_salary|employee_residence|
+----------------+--------------+------------------+
|            NULL|        250000|                US|
|            NULL|        600000|                US|
|          middle|        450000|                US|
|          senior|        412000|                US|
+----------------+--------------+------------------+



write df_new_best like 1.csv and load then it to df_final

In [82]:
df_new_best.write.mode('overwrite').options(header='True', delimiter=',').csv('1.csv')

In [126]:
df_final = spark.read.csv('1.csv', header=True)

In [127]:
df_final = df_final.withColumnRenamed('experience_level', 'exp_lvl')

print df_final

In [128]:
df_final.show()

+-------+--------------+------------------+
|exp_lvl|biggest_salary|employee_residence|
+-------+--------------+------------------+
|   NULL|        250000|                US|
|   NULL|        600000|                US|
| middle|        450000|                US|
| senior|        412000|                US|
+-------+--------------+------------------+



filter df_final to delete experience_level where it Null, then join this table by biggest_salary(salary_in_usd) and employee_residence with entire df

In [130]:
df_final = df_final.na.drop(subset=['exp_lvl']).join(df2, (df_final['biggest_salary'] == df2['salary_in_usd']) &
                                                              (df_final['employee_residence'] == df2['employee_residence']),
                                                              how='inner')       

print df_final

In [131]:
df_final.show()

+-------+--------------+------------------+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|exp_lvl|biggest_salary|employee_residence| id|work_year|experience_level|employment_type|           job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+-------+--------------+------------------+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
| middle|        450000|                US| 33|     2020|              MI|             FT|  Research Scientist|450000|            USD|       450000|                US|           0|              US|           M|
| senior|        412000|                US| 63|     2020|              SE|             FT|      Data Scientist|412000|            USD|       412000|        

In [132]:
df_final.toPandas()

,exp_lvl,biggest_salary,employee_residence,id,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,middle,450000,US,33,2020,MI,FT,Research Scientist,450000,USD,450000,US,0,US,M
1,senior,412000,US,63,2020,SE,FT,Data Scientist,412000,USD,412000,US,100,US,L
2,middle,450000,US,97,2021,MI,FT,Financial Data Analyst,450000,USD,450000,US,100,US,L


last task is to save in variable and then print this variable of the biggest salary_in_usd from df_final

In [141]:
max_salary_df_final = df_final.agg({'salary_in_usd': 'max'}).collect()[0][0]

In [143]:
print(f'Biggest salary in usd from df_final: {max_salary_df_final}')

Biggest salary in usd from df_final: 450000


It is the end of PySpark basics. In other lessons you will learn optimizations technics and how to make distributed system